<a href="https://colab.research.google.com/github/jxhn13/rag_qa/blob/main/ragqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
'''
# **RAG QA Bot using Pinecone and Cohere**
This notebook demonstrates a Retrieval-Augmented Generation (RAG) model that uses Pinecone for vector storage and Cohere for generating embeddings and responses. The goal is to upload documents, embed them into vectors, store them in Pinecone, and answer questions based on those documents using Cohere.

## **Installation**

Make sure to install the required libraries:

```python'''
!pip install pinecone-client cohere


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.4/388.4 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8

In [6]:
import pinecone
import cohere
import os

# Set your API keys (you should replace 'your-pinecone-api-key' and 'your-cohere-api-key')
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY", '')
COHERE_API_KEY = os.getenv("COHERE_API_KEY", '')

# Initialize the Cohere client
cohere_client = cohere.Client(COHERE_API_KEY)

# Initialize Pinecone using Pinecone class
pinecone.init(api_key=PINECONE_API_KEY, environment="us-east-1") # Instead of pinecone.init, use:
#pinecone_client = pinecone.Pinecone(api_key=PINECONE_API_KEY, environment="us-east-1")

AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [7]:
from google.colab import files

# Function to upload and process files
def upload_file():
    uploaded = files.upload()  # User can upload files
    for filename in uploaded:
        print(f"Processing {filename}...")
        # You can add logic here to handle the document text


In [8]:
# Function to create embeddings using Cohere and store them in Pinecone
def create_embeddings(document_text):
    # Generate embeddings from document text
    response = cohere_client.embed(texts=[document_text])
    embedding = response.embeddings[0]

    # Store the embedding in Pinecone
    pinecone_index = pinecone.Index("your-index-name")
    pinecone_index.upsert([(f"doc-{hash(document_text)}", embedding)])

    print("Embedding created and stored successfully!")


In [9]:
# Function to answer questions using RAG (Retrieval-Augmented Generation)
def ask_question(question):
    # Retrieve relevant documents from Pinecone
    query_embedding = cohere_client.embed(texts=[question]).embeddings[0]
    pinecone_index = pinecone.Index("your-index-name")

    # Search Pinecone for relevant documents
    results = pinecone_index.query(queries=[query_embedding], top_k=3)

    # Use Cohere to generate an answer based on the retrieved documents
    context = " ".join([res["metadata"]["text"] for res in results["matches"]])
    response = cohere_client.generate(prompt=f"Question: {question}\nContext: {context}\nAnswer:")

    return response.generations[0].text

# Example usage:
question = "What is the main point of document X?"
answer = ask_question(question)
print(f"Answer: {answer}")


TypeError: Index.__init__() missing 1 required positional argument: 'host'